In [115]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
import numpy



In [116]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [118]:
# store CSV into DataFrame
# Rename columns to fit the tables created in the database
# Did an loc to retrieve only movie type
csv_file = "Film_Industry_Research_Project/LaQuita/Resources/netflix_titles.csv"
movies_data = pd.read_csv(csv_file)
# movies_data = movies_data.rename(columns={"cast": "casts"})
# movies_data = movies_data.loc[movies_data["type"]=="Movie"]
movies_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [119]:
# create new data with selected columns
new_movies_data = movies_data[['title', 'type', 'country']].copy()
new_movies_data
new_movies_data.head()

,title,type,country
0,Dick Johnson Is Dead,Movie,United States
1,Blood & Water,TV Show,South Africa
2,Ganglands,TV Show,NaN
3,Jailbirds New Orleans,TV Show,NaN
4,Kota Factory,TV Show,India


In [120]:
# Store JSON data into a DataFrame
# Rename columns to fit the tables created in the database
json_file = "Film_Industry_Research_Project/LaQuita/Resources/movie_revenue.json"
movie_revenue = pd.read_json(json_file)
movie_revenue = movie_revenue.rename(columns={"Movie Title": "title", "Worldwide Gross ($)": "worldwide_gross"})
movie_revenue.head()

,Rank,Release Date,title,Production Budget ($),worldwide_gross,Domestic Gross ($)
0,5293,8/2/1915,The Birth of a Nation,"$110,000","$11,000,000","$10,000,000"
1,5140,5/9/1916,Intolerance,"$385,907",$0,$0
2,5230,12/24/1916,"20,000 Leagues Under the Sea","$200,000","$8,000,000","$8,000,000"
3,5299,9/17/1920,Over the Hill to the Poorhouse,"$100,000","$3,000,000","$3,000,000"
4,5222,1/1/1925,The Big Parade,"$245,000","$22,000,000","$11,000,000"


In [121]:
# Clean DataFrame by selecting only columns that are relevant 
new_movie_revenue = movie_revenue[['title', 'worldwide_gross']].copy()
new_movie_revenue.head()

,title,worldwide_gross
0,The Birth of a Nation,"$11,000,000"
1,Intolerance,$0
2,"20,000 Leagues Under the Sea","$8,000,000"
3,Over the Hill to the Poorhouse,"$3,000,000"
4,The Big Parade,"$22,000,000"


In [124]:
# rename both DataFrame to merge them together by title 
revenue = pd.merge(new_movies_data, new_movie_revenue, on='title', how='inner')
revenue.head()

,title,type,country,worldwide_gross
0,Grown Ups,Movie,United States,"$272,223,430"
1,Dark Skies,Movie,United States,"$27,704,111"
2,Paranoia,Movie,"United States, India, France","$16,173,593"
3,Jaws,Movie,United States,"$470,700,000"
4,Jaws 2,Movie,United States,"$208,900,376"


In [125]:
connection_string = "postgres:Treasure#444@localhost:5432/movie_revenue_db"
engine = create_engine(f'postgresql://{connection_string}')

In [126]:
# check for tables
engine.table_names()


['revenue', 'movie_revenue_db']

In [127]:
revenue.to_sql(name='movie_revenue_db', con=engine, if_exists='append', index=True)